```` ..English translation in progress..````

Die SQL Datenbank sowie die CSV-Datei für die Bearbeitung der nachfolgenden Aufgaben können [hier](https://www.dropbox.com/s/iislkxvmpklqqj5/sqlassignment_data.zip?dl=1) heruntergeladen werden.

# Aufgaben

1. Stellen Sie eine Verbindung zur Datenbank (laender.db) her. Produzieren Sie jeweils eine Liste mit den Namen der Tabellen und den Namen der Spalten in den Tabellen, inklusive deren Datentypen. Erklären Sie kurz die Datentypen und deren Äquivalente in Python. Hinweis: In der SQLite Matertabelle finden Sie die entsprechenden Daten.
2. Öffnen sie die Datei armutsgefahr.txt in Python und geben Sie den Inhalt als einen String wieder (Lösung muss keinen Output enthalten). Um welches Datenschemata handelt es sich hierbei am wahrscheinlichsten (Begründung)?
3. Erzeugen Sie eine neue Tabelle in laender.db, mit dem Namen "armutsgefahr" und drei Spalten ('land', 'jahr', 'quote'). Befüllen Sie die Tabelle mit den entsprechenden Daten aus armutsgefahr.txt.
4. Führen Sie eine SQL-Abfrage der Datenbank nach allen denjenigen Regierungschefs durch, an deren Reigierungen die SPD beteiligt ist. Grenzen Sie diese Abfrage in einem zweiten Schritt auf die alten Bundesländer ein.
5. Produzieren Sie eine Abfrage, die folgende Elemente enthält: 
    * Das Stimmgewicht des jeweiligen Landes;
    * Die absolute Differenz der Einwohnerzahl in der Tabelle laender_info und der Einwohnerzahl(gesamt) in der Tabelle migrationshintergrund_2013; 
    * Prozentualer Anteil der Personen mit Migrationshintergrund an der Gesamtbevölkerung (Tabelle: migrationshintergrund_2013);
    * Eingegrenzt auf diejenigen Länder, die mehr als 200 Einwohner pro km$^2$ aber nicht mehr als 2000 haben;
6. Ersetzen Sie sämtliche fehlenden Werte in der Spalte hauptstadt der Tabelle laender_info.

# Aufgabenbearbeitung

## Teilaufgabe 1

Um Informationen aus der SQLite-Datenbank extrahieren zu können werden zunächst das Arbeitsverzeichnis gewählt, das benötigte Programmpaket geladen sowie eine Verbindung zur Datenbank aufgebaut. Zur Ausführung von SQLite Befehlen muss zusätzlich ein Cursor Objekt erstellt werden.    

In [1]:
%cd "E:\Dropbox\Soziologie Master\Medium Data\Aufgaben\Aufgabe 1"
import sqlite3

conn = sqlite3.connect('laender.db')
c = conn.cursor()

E:\Dropbox\Soziologie Master\Medium Data\Aufgaben\Aufgabe 1


Über die SQLite Mastertabelle werden die Tabellenbezeichnungen ausgegeben und in einer Liste gespeichert. Da die Bezeichnungen in Form von Tupeln vorliegen, wird die Liste zusätzlich bearbeitet, sodass anschließend gemäß der Aufgabenstellung nur noch die Strings für die Tabellennamen enthalten sind.

In [2]:
tables = c.execute('''SELECT name FROM sqlite_master\
                      WHERE type='table' ''').fetchall()

tables = [table[0] for table in tables]
print tables

[u'laender_info', u'migrationshintergrund_2013']


Anschließend werden die Informationen zu den beiden Tabellen ausgelesen und über List-Comprehensions die jeweiligen Spaltenbezeichnungen inklusive deren Datentypen in Form von 2er-Tupeln in weitere Listen geschrieben.

In [3]:
countries = c.execute('''PRAGMA table_info(laender_info)''').fetchall()
countries[:2]

[(0, u'index', u'INTEGER', 0, None, 0), (1, u'land', u'TEXT', 0, None, 0)]

In [4]:
country_cols = [(col[1], col[2]) for col in countries]
country_cols

[(u'index', u'INTEGER'),
 (u'land', u'TEXT'),
 (u'kuerzel', u'TEXT'),
 (u'hauptstadt', u'TEXT'),
 (u'beitrittsdatum', u'INTEGER'),
 (u'regierungschef', u'TEXT'),
 (u'regierungspartei', u'TEXT'),
 (u'stimmgewicht', u'INTEGER'),
 (u'flaeche', u'INTEGER'),
 (u'einwohner_mio', u'REAL'),
 (u'einwohner_km2', u'INTEGER'),
 (u'auslaender', u'REAL'),
 (u'sprachen', u'TEXT')]

In [5]:
migration = c.execute('''PRAGMA table_info(migrationshintergrund_2013)''').fetchall()
migration[:2]

[(0, u'index', u'INTEGER', 0, None, 0), (1, u'land', u'TEXT', 0, None, 0)]

In [6]:
mig_cols = [(col[1], col[2]) for col in migration]
mig_cols

[(u'index', u'INTEGER'),
 (u'land', u'TEXT'),
 (u'insgesamt', u'INTEGER'),
 (u'ohne_mh', u'INTEGER'),
 (u'mit_mh', u'INTEGER')]

In einem weiteren Schritt werden die Listen mit den Tupeln der Spalteninformationen zu einer gemeinsamen Liste zusammengeschrieben. *Anmerkung*: Dieser Schritt hätte für effizienteren Code auch bereits in der vorherigen Aufbereitung durchgeführt werden können, wurde aus Übersichtsgründen aber erst an dieser Stelle ausgeführt.

In [9]:
table_cols = country_cols + mig_cols
table_cols

[(u'index', u'INTEGER'),
 (u'land', u'TEXT'),
 (u'kuerzel', u'TEXT'),
 (u'hauptstadt', u'TEXT'),
 (u'beitrittsdatum', u'INTEGER'),
 (u'regierungschef', u'TEXT'),
 (u'regierungspartei', u'TEXT'),
 (u'stimmgewicht', u'INTEGER'),
 (u'flaeche', u'INTEGER'),
 (u'einwohner_mio', u'REAL'),
 (u'einwohner_km2', u'INTEGER'),
 (u'auslaender', u'REAL'),
 (u'sprachen', u'TEXT'),
 (u'index', u'INTEGER'),
 (u'land', u'TEXT'),
 (u'insgesamt', u'INTEGER'),
 (u'ohne_mh', u'INTEGER'),
 (u'mit_mh', u'INTEGER')]

Innerhalb der Tabellen finden sich drei verschiedene Datentypen:

* INTEGER: umfassen *ganze Zahlen* und entsprechen dem gleichnamigen Python Äquivalent. Je nach Länge der Zahl wird der Wert in Python allerdings möglicherweise ins *Long* Format konvertiert. 
* REAL: umfassen *Gleitkommazahlen* und entsprechen dem Python-Datentyp *float*. 
* TEXT: umfassen eine Abfolge von *Zeichen und/oder Symbolen* und entsprechen dem Python-Datentyp *String*. Als Standardeinstellung wird *Unicode-Enkodierung* verwendet.

## Teilaufgabe 2

Zum Öffnen der Datei wird eine Verbindung mit dem Dateiobjekt im Lesemodus erstellt und der Inhalt des Objekts anschließend als Gesamtstring eingelesen. Zur Erkennung des Datenschemas wird anschließend ein Teil des Strings ausgegeben.

In [23]:
with open('armutsgefahr.txt', 'r') as f:
    finput = f.read()
    
print finput[:400]

Bundesland	2005	2006	2007	2008	2009	2010	2011	2012	2013
Deutschland	14.7	14.0	14.3	14.4	14.6	14.5	15.0	15.0	15.5
Altes Bundesgebiet ohne Berlin	13.2	12.7	12.9	13.1	13.3	13.3	13.8	13.9	14.4
Neue Bundeslaender mit Berlin	20.4	19.2	19.5	19.5	19.5	19.0	19.4	19.6	19.8
Baden-Wuerttemberg	10.6	10.1	10.0	10.2	10.9	11.0	11.1	11.1	11.4
Bayern	11.4	10.9	11.0	10.8	11.1	10.8	11.1	11.0	11.3
Berlin	19.7	17.0	17.


Zwar sind die Informationen im Text-Format *.txt* abgespeichert, die Struktur der abgebildeten Daten lässt aber auf eine *.csv*-Formatierung  schließen. Im CSV (*Comma-Separated-Value*) Format wird mit Hilfe eines Delimiters zwischen verschiedenen Werten unterschieden. Gemäß der Bezeichnung des Formats werden hierfür häufig Kommata eingesetzt. Im hier vorliegenden Fall werden einzelne Werte jedoch über Tabulatoren abgegrenzt, sodass Zeilenumbrüche ** \n ** Zeilen und  Tabulatoren ** \t ** Spalten definieren.

## Teilaufgabe 3

Die geforderte Tabelle wird zunächst in der Datenbank angelegt. Da zum Primärschlüssel der Tabelle keine Bedingungen definiert sind, wird dieser für die Aufgabe manuell festgelegt, generiert und integriert.

In [5]:
c.execute('''CREATE TABLE armutsgefahr
             (pid INTEGER PRIMARY KEY,
             land TEXT,
              jahr INTEGER,
              quote REAL)''')

Um die Tabelle mit den Informationen Zu Land, Jahr und Armutsquote zu befüllen, müssen die Daten aus der Textdatei zunächst in einer verwertbaren Form in Python eingelesen werden. Wie vorher festgestellt wurde, liegen die Daten im *.csv* Format vor, weshalb ein entsprechender Reader verwendet werden kann der den Input zeilenweise in eine Liste schreibt.

In [63]:
import csv
import itertools

data = []
with open('armutzsgefahr.txt', 'r') as f:
     reader = csv.reader(f, delimiter='\t')
     for row in reader:
        data.append(row)

Im nachfolgenden Codeblock werden mehrere Anpassungen der Datenstruktur durchgeführt. Zunächst werden einzeln Länder,Jahre und Armutsquote abgespeichert sowie ein Primärschlüssel festgelegt, dessen Wertemenge sich aus dem Produkt der Anzahl der Länder und der Anzahl der Jahre ergibt. Für jede Kombination ist demnach ein eindeutiger Primärschlüssel vorhanden.    
Die Länder- und Jahresinformationen werden anschließend über ein Tupel des Produkts der beiden Listen und List-Comprehensions so aufbereitet, dass jedes Land neun mal (Anzahl der Jahre) und jedes Jahr 19 mal (Anzahl der Länder) vorliegt.   
Anschließend können die Informationen über die zip-Funktion zusammengespielt werden.

In [ ]:
laender = [row[0] for row in data[1:]]
quote = [value for row in data[1:]for value in row[1:]]
jahre = data[0][1:]
pid = range(1, len(laender)*len(jahre)+1)

tuples = list(itertools.product(laender,jahre))
land_dup = [val[0] for val in tuples]
jahr_dup = [val[1] for val in tuples]

combined =zip(pid, land_dup, jahr_dup, quote)

Der Nachfolgende Output zeigt zur Überprüfung die Längen der verwendeten Listen sowie einen Auszug aus den kombinierten Informationen.

In [102]:
print len(laender), len(jahre), len(laender) * len(jahre),  len(pid),
combined[:11]

19 9 171 171

[(1, 'Deutschland', '2005', '14.7'),
 (2, 'Deutschland', '2006', '14.0'),
 (3, 'Deutschland', '2007', '14.3'),
 (4, 'Deutschland', '2008', '14.4'),
 (5, 'Deutschland', '2009', '14.6'),
 (6, 'Deutschland', '2010', '14.5'),
 (7, 'Deutschland', '2011', '15.0'),
 (8, 'Deutschland', '2012', '15.0'),
 (9, 'Deutschland', '2013', '15.5'),
 (10, 'Altes Bundesgebiet ohne Berlin', '2005', '13.2'),
 (11, 'Altes Bundesgebiet ohne Berlin', '2006', '12.7')]

Im letzten Schritt können die Daten direkt über die Zuspielung der kombinierten Liste in einer SQL-Abfrage in die Tabelle aufgenommen werden.

In [98]:
c.executemany('''INSERT INTO armutsgefahr VALUES (?,?,?,?)''', 
              combined)

Zur Überprüfung wird hier noch einmal der Output aus der SQL-Tabelle ausgegeben, der sowohl mit den in Python aufbereiteten Daten als auch mit den Rohdaten aus der Textdatei übereinstimmt. Die Strings wurden bei der Integrierung automatisch in Unicode enkodiert.

In [101]:
c.execute('''SELECT * FROM armutsgefahr''').fetchall()[:11]

[(1, u'Deutschland', 2005, 14.7),
 (2, u'Deutschland', 2006, 14.0),
 (3, u'Deutschland', 2007, 14.3),
 (4, u'Deutschland', 2008, 14.4),
 (5, u'Deutschland', 2009, 14.6),
 (6, u'Deutschland', 2010, 14.5),
 (7, u'Deutschland', 2011, 15.0),
 (8, u'Deutschland', 2012, 15.0),
 (9, u'Deutschland', 2013, 15.5),
 (10, u'Altes Bundesgebiet ohne Berlin', 2005, 13.2),
 (11, u'Altes Bundesgebiet ohne Berlin', 2006, 12.7)]

## Teilaufgabe 4

Für die korrekte Abfrage müssen Informationen aus den Spalten *regierungschef*, *regierungspartei* und *land* aus der Tabelle *laender_info* zusammengespielt werden. Für den ersten Schritt wird hierfür der SQlite Befehl *LIKE* verwendet, mit dem innerhalb von Strings nach Substrings gesucht werden kann. Aus der Spalte *regierungspartei* können somit alle Einträge ausgewählt werden in denen sich der String "*SPD*" befindet.

In [110]:
c.execute('''SELECT regierungschef FROM laender_info
             WHERE regierungspartei LIKE '%SPD%' ''').fetchall()

[(u'Jens Boehrnsen (SPD)',),
 (u'Olaf Scholz (SPD)',),
 (u'Klaus Wowereit (SPD)',),
 (u'Stanislaw Tillich (CDU)',),
 (u'Reiner Haseloff (CDU)',),
 (u'Annegret Kramp-Karrenbauer (CDU)',),
 (u'Torsten Albig (SPD)',),
 (u'Bodo Ramelow (Die Linke)',),
 (u'Malu Dreyer (SPD)',),
 (u'Erwin Sellering (SPD)',),
 (u'Dietmar Woidke (SPD)',),
 (u'Hannelore Kraft (SPD)',),
 (u'Winfried Kretschmann (Gruene)',),
 (u'Stephan Weil (SPD)',)]

Für die zusätzliche Selektion nach Bundesregion wird zunächst eine Liste der neuen Bundesländer (Berlin eingeschlossen) angelegt. In der SQL Abfrage wird dann mit Hilfe von Platzhaltern auf diese Liste zurückgegriffen, um zusätzlich zur ersten Bedingung nur Regierungschefs ausgeben zu lassen, die **nicht** in den neuen Bundesländern regiert haben. Die Negierung ist hier effektiv, da mehr alte als neue Bundesländer vorhanden sind und das Anlegen einer Liste für alte Bundesländer mehr Code umfasst hätte.

In [153]:
ost = ['Brandenburg', 'Berlin', 'Thueringen', 'Sachsen',\
       'Sachsen-Anhalt', 'Mecklenburg-Vorpommern']

In [154]:
c.execute('''SELECT regierungschef FROM laender_info \
             WHERE regierungspartei LIKE '%SPD%'\
             AND land NOT IN (?, ?, ?, ?, ?, ?) ''', ost).fetchall()

[(u'Jens Boehrnsen (SPD)',),
 (u'Olaf Scholz (SPD)',),
 (u'Annegret Kramp-Karrenbauer (CDU)',),
 (u'Torsten Albig (SPD)',),
 (u'Malu Dreyer (SPD)',),
 (u'Hannelore Kraft (SPD)',),
 (u'Winfried Kretschmann (Gruene)',),
 (u'Stephan Weil (SPD)',)]

##  Teilaufgabe 5

Zunächst müssen zur Umsetzung der korrekten Abfrage die benötigten Spalten aus den beiden Tabellen ausfindig gemacht werden:

    stimmgewicht    
    insgesamt
    ohne_mh    
    mit_mh    
    einwohner_mio    
    einwohner_km2    

Die nachfolgende Inspektion der Daten lässt vermuten, dass die Einwohnerzahl in der Tabelle *laender_info* in Millionen, in der Tabelle *migrationshintergrund_2013* allerdings in Tausend hinterlegt ist, was bei der Umsetzung der Abfrage entsprechend berücksichtigt wird.

In [104]:
c.execute('''
    SELECT laender_info.land, stimmgewicht, einwohner_mio, \
    insgesamt, ohne_mh, mit_mh, einwohner_km2 \
    FROM laender_info \
    NATURAL JOIN migrationshintergrund_2013 ''').fetchall()

[(u'Bremen', 3, 0.652, 655, 468, 187, 1576),
 (u'Hamburg', 3, 2.0, 1744, 1239, 504, 2366),
 (u'Berlin', 4, 3.0, 3396, 2493, 903, 3829),
 (u'Sachsen', 4, 4.0, 4044, 3854, 190, 225),
 (u'Sachsen-Anhalt', 4, 2.0, 2250, 2152, 97, 114),
 (u'Saarland', 3, 0.998, 992, 821, 172, 396),
 (u'Bayern', 6, 12.0, 12556, 9967, 2589, 178),
 (u'Schleswig-Holstein', 4, 3.0, 2810, 2458, 352, 179),
 (u'Thueringen', 4, 2.0, 2164, 2076, 88, 138),
 (u'Rheinland-Pfalz', 4, 4.0, 3991, 3168, 823, 202),
 (u'Hessen', 5, 6.0, 6029, 4351, 1678, 287),
 (u'Mecklenburg-Vorpommern', 3, 2.0, 1598, 1527, 71, 71),
 (u'Brandenburg', 4, 2.0, 2448, 2326, 123, 85),
 (u'Nordrhein-Westfalen', 6, 18.0, 17553, 13137, 4415, 523),
 (u'Baden-Wuerttemberg', 6, 11.0, 10599, 7644, 2955, 301),
 (u'Niedersachsen', 6, 8.0, 7784, 6393, 1391, 166)]

   
Für die Abfrage wird ein *natural join* durchgeführt um die Informationen aus den beiden Tabellen über die Indizierung zusammenzuspielen. Neben der Eingrenzung nach Einwohnern pro km² wird für den prozentualen Anteil der Personen mit Migrationshintergrund zusätzlich der Datentyp eines Attributs durch die SQL-Funktion *cast* transformiert.    
Für eine einfachere Zuordnung ist in der Abfrage neben den geforderten Elementen zusätzlich auch das jeweilige Land aufgeführt.

In [26]:
c.execute('''
    SELECT stimmgewicht, laender_info.land, ABS(einwohner_mio * 1000 - insgesamt),\
    CAST(mit_mh AS REAL)/insgesamt from laender_info \
    NATURAL JOIN migrationshintergrund_2013 \
    WHERE einwohner_km2 > 200 AND einwohner_km2 <= 2000 \ ''').fetchall()

[(3, u'Bremen', 3.0, 0.28549618320610687),
 (4, u'Sachsen', 44.0, 0.046983184965380814),
 (3, u'Saarland', 6.0, 0.17338709677419356),
 (4, u'Rheinland-Pfalz', 9.0, 0.20621398145828113),
 (5, u'Hessen', 29.0, 0.2783214463426771),
 (6, u'Nordrhein-Westfalen', 447.0, 0.25152395601891414),
 (6, u'Baden-Wuerttemberg', 401.0, 0.27879988678177187)]

Auffällig sind hier insbesondere die hohen Differenzen der Einwohnerzahlen für Nordrhein-Westfalen und Baden-Wuerttemberg. Die oben dargestellte Dateninspektion lässt vermuten, dass die Differenzen sich aus Rundungen der Einwohnerzahlangaben in Millionen ergeben - der Datentyp *real* wurde hier nicht in Form von Nachkommastellen ausgenutzt.

## Teilaufgabe 6

Zunächst werden die Einträge ausgemacht, in denen keine Informationen zur Hauptstadt vorhanden sind und in einer Liste abgespeichert.

In [2]:
staedte = [stadt[0] for stadt in c.execute\
        ('''SELECT land FROM laender_info WHERE hauptstadt IS NULL ''')]

In [3]:
staedte

[u'Hamburg', u'Berlin']

Es handelt sich hier um die beiden Stadtstaaten *Berlin* und *Hamburg*, für welche die fehlenden Informationen direkt aus der Spalte *land* übernommen werden können. Die fehlenden Werte werden dementsprechend ersetzt.. 

In [4]:
## UPDATE TABLE
c.executescript('''
    UPDATE laender_info SET hauptstadt='Hamburg' \
    WHERE land =='Hamburg';
    UPDATE laender_info SET hauptstadt='Berlin' \
    WHERE land =='Berlin' ''')

..und die neuen Werte zuletzt über die Verwendung der zuvor generierten Liste ausgegeben.

In [5]:
c.execute('''SELECT hauptstadt FROM laender_info\
             WHERE land IN (?,?) ''', staedte).fetchall()

[(u'Hamburg',), (u'Berlin',)]